In [ ]:

import pandas as pd
import sys
from pathlib import Path
import os
import json
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import STL


# adiciona a pasta src ao sys.path
sys.path.append(str(Path.cwd().parent.parent / "src")) # Necessário contar quantos niveis subir nas pastas, incluir 1 .parent para cada nivel.

# Importa o script paths de utils para geracão de caminhos absolutos
from utils.paths import resolve_env_path, PROJECT_ROOT
from classes.time_series_dataset import TimeSeriesDataset
from classes.plotter import Plotter
from utils.calculations import plot_stl_decomposition
from utils.calculations import process_df

# Utilidades para modelagem
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import evaluate
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut, MinTrace
from hierarchicalforecast.utils import aggregate, HierarchicalPlot
from utilsforecast.losses import rmse, mae, smape, mase, scaled_crps, mqloss


# Adiciona caminhos importantes
ARRIVALS_PATH = resolve_env_path("ARRIVALS_PATH")

# Selecting seaborn theme
sns.set_theme(
    style="ticks",
    palette="pastel",
    context="notebook"
)


Reading clean dataset


In [5]:
arrivals_dataset = TimeSeriesDataset(ARRIVALS_PATH)
df = arrivals_dataset.get_data_without_missing_values()

Entry route codes

In [7]:
arrivals_dataset.entry_route_table

,entry_route,entry_route_id
0,terrestre,2
1,fluvial,4
2,aereo,1
3,maritimo,3


Continent codes

In [10]:
arrivals_dataset.continent_table

,continent,continent_id
0,africa,1
1,america_central_e_caribe,2
2,america_do_norte,3
3,america_do_sul,4
4,asia,5
5,europa,6
6,oceania,7
7,continente_nao_especificado,8


In [9]:
df

,continent,continent_id,country,country_id,state,state_id,entry_route,entry_route_id,year,month,month_id,arrivals,file_index,date
0,africa,1,africa_do_sul,2,acre,1,terrestre,2,2015,janeiro,1,2.0,file_0,2015-01-01
1,africa,1,angola,6,acre,1,terrestre,2,2015,janeiro,1,0.0,file_0,2015-01-01
2,africa,1,cabo_verde,35,acre,1,terrestre,2,2015,janeiro,1,0.0,file_0,2015-01-01
3,africa,1,egito,60,acre,1,terrestre,2,2015,janeiro,1,0.0,file_0,2015-01-01
4,africa,1,gana,77,acre,1,terrestre,2,2015,janeiro,1,0.0,file_0,2015-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762567,america_do_sul,4,venezuela,238,sao_paulo,25,maritimo,3,2018,agosto,8,0.0,file_24,2018-08-01
762568,america_do_sul,4,venezuela,238,sao_paulo,25,maritimo,3,2018,setembro,9,0.0,file_24,2018-09-01
762569,america_do_sul,4,venezuela,238,sao_paulo,25,maritimo,3,2018,outubro,10,0.0,file_24,2018-10-01
762570,america_do_sul,4,venezuela,238,sao_paulo,25,maritimo,3,2018,novembro,11,0.0,file_24,2018-11-01


## Total arrivals prediction

,continent,continent_id,country,country_id,state,state_id,entry_route,entry_route_id,year,month,month_id,arrivals,file_index,date
0,africa,1,africa_do_sul,2,acre,1,terrestre,2,2015,janeiro,1,2.0,file_0,2015-01-01
1,africa,1,angola,6,acre,1,terrestre,2,2015,janeiro,1,0.0,file_0,2015-01-01
2,africa,1,cabo_verde,35,acre,1,terrestre,2,2015,janeiro,1,0.0,file_0,2015-01-01
3,africa,1,egito,60,acre,1,terrestre,2,2015,janeiro,1,0.0,file_0,2015-01-01
4,africa,1,gana,77,acre,1,terrestre,2,2015,janeiro,1,0.0,file_0,2015-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762567,america_do_sul,4,venezuela,238,sao_paulo,25,maritimo,3,2018,agosto,8,0.0,file_24,2018-08-01
762568,america_do_sul,4,venezuela,238,sao_paulo,25,maritimo,3,2018,setembro,9,0.0,file_24,2018-09-01
762569,america_do_sul,4,venezuela,238,sao_paulo,25,maritimo,3,2018,outubro,10,0.0,file_24,2018-10-01
762570,america_do_sul,4,venezuela,238,sao_paulo,25,maritimo,3,2018,novembro,11,0.0,file_24,2018-11-01
